In [1]:
# https://www.quantinsti.com/blog/deep-learning-artificial-neural-network-tensorflow-python

In [2]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as pl
import talib # technical analysis library
import random

In [3]:
dataset = pd.read_csv("./dataset/AAPL.csv")
dataset = dataset.dropna()
dataset = dataset[['Open','High','Low','Close']]

In [4]:
dataset['H-L'] = dataset['High'] - dataset['Low']
dataset['O-C'] = dataset['Close'] - dataset['Open']
dataset['3day MA'] = dataset['Close'].shift(1).rolling(window=3).mean()
dataset['10day MA'] = dataset['Close'].shift(1).rolling(window=10).mean()
dataset['30day MA'] = dataset['Close'].shift(1).rolling(window=30).mean()
dataset['Std_dev'] = dataset['Close'].rolling(5).std()
dataset['RSI'] = talib.RSI(dataset['Close'].values, timeperiod=9)
dataset['Williams %R'] = talib.WILLR(dataset['High'].values, dataset['Low'].values, dataset['Close'].values, 7)

In [5]:
dataset['Price_Rise'] = np.where(dataset['Close'].shift(-1)>dataset['Close'], 1, 0)

In [6]:
dataset = dataset.dropna()

In [7]:
# remove OHLC
data = dataset.iloc[:, 4:]

In [8]:
# conver numpy
n = data.shape[0]
p = data.shape[1]
data = data.values

In [9]:
# training and testing
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end+1
test_end = n
data_train = data[np.arange(train_start, train_end), :]
data_test = data[np.arange(test_start, test_end),:]

In [10]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

In [11]:
X_train = data_train[:, 0:-1]
y_train = data_train[:, -1]
X_test = data_test[:, 0:-1]
y_test = data_test[:, -1]

In [12]:
# Artificial Neural Network
n_features =X_train.shape[1] 
n_neurons_1 = 512
n_neurons_2 = 256
n_neurons_3 = 128
net = tf.InteractiveSession()

In [13]:
X = tf.placeholder(dtype=tf.float32, shape=[None, n_features])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

In [14]:
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode='fan_avg', distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [15]:
# neural network design
W_hidden_1 = tf.Variable(weight_initializer([n_features, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))

W_out = tf.Variable(weight_initializer([n_neurons_3, 1]))
bias_out = tf.Variable(bias_initializer([1]))

In [16]:
# neural network composition
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
out = tf.transpose(tf.add(tf.matmul(hidden_3, W_out), bias_out))

In [17]:
mse = tf.reduce_mean(tf.squared_difference(out, Y))

In [18]:
opt = tf.train.AdamOptimizer().minimize(mse)

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[node Variable/read (defined at <ipython-input-15-a12056a16906>:2)  = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable)]]

Caused by op 'Variable/read', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sharavsambuu/src/stock-price-prediction-exp/env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sharavsambuu/src/stock-price-prediction-exp/env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sharavsambuu/src/stock-price-prediction-exp/env/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/shara